In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd

In [0]:
import glob
image_list = glob.glob('/content/gdrive/My Drive/VisDrone2019-DET-val/VisDrone2019-DET-val/images/*.jpg')

In [0]:
txt_list = glob.glob('/content/gdrive/My Drive/VisDrone2019-DET-val/VisDrone2019-DET-val/annotations/*.txt')

In [5]:
import torch 
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')
ssd_model.to('cuda')
ssd_model.eval()

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/archive/torchhub.zip" to /root/.cache/torch/hub/torchhub.zip
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [0]:
uris = image_list
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs, precision == 'fp16')

In [0]:
with torch.no_grad():
    detections_batch = ssd_model(tensor)

In [8]:
results_per_input = utils.decode_results(detections_batch)
best_results_per_input = [utils.pick_best(results, 0.00) for results in results_per_input]
classes_to_labels = utils.get_coco_object_dictionary()

In [0]:
import pandas as pd
import numpy as np

In [0]:
def annots(txt):
  data = []
  data1 = []
  DATA = pd.DataFrame()
  for f in txt:
    data.append(np.genfromtxt(f, delimiter=',', dtype=np.float))
    data1.append(f[f.rindex('/')+1:])
  for i in range(len(data1)):
    df = pd.DataFrame(np.matrix(data[i]),columns=['bbox_left','bbox_top','bbox_width','bbox_height','score','object_category','truncation','occlusion'])
    df2 = pd.Series(np.full(len(np.matrix(data[i])),data1[i]))  
    df['name_file'] = df2
    DATA = DATA.append(df, ignore_index = True)
  return DATA

In [0]:
df_annots = annots(txt_list)

In [0]:
df_annots.to_csv('/content/gdrive/My Drive/df_annots.csv', index = False, header=True)

In [13]:
df_annots

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file
0,879.0,483.0,10.0,31.0,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt
1,854.0,479.0,8.0,18.0,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt
2,620.0,445.0,63.0,35.0,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt
3,579.0,406.0,61.0,30.0,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt
4,48.0,489.0,104.0,42.0,1.0,4.0,0.0,2.0,0000249_00001_d_0000001.txt
...,...,...,...,...,...,...,...,...,...
40164,709.0,634.0,117.0,51.0,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt
40165,1046.0,577.0,101.0,43.0,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt
40166,1232.0,580.0,103.0,45.0,1.0,4.0,0.0,1.0,0000316_01201_d_0000525.txt
40167,1305.0,519.0,54.0,35.0,1.0,8.0,1.0,1.0,0000316_01201_d_0000525.txt


In [0]:
def func_annots(txt, best_results):
  data = []
  data1 = []
  DATA = pd.DataFrame()
  for f in txt:
    data1.append(f[f.rindex('/')+1:])
  for i in range(len(data1)):
    for j in range(len(best_results[i][0])):
      left = best_results[i][0][j][0]
      bot = best_results[i][0][j][1]
      right = best_results[i][0][j][2]
      top = best_results[i][0][j][3]
      best_results[i][0][j][0] = 300 * left
      best_results[i][0][j][1] = 300 * bot
      best_results[i][0][j][2] = 300 * (right - left)
      best_results[i][0][j][3] = 300 * (top - bot)
    df1 = pd.DataFrame(best_results[i][0], columns=['bbox_left','bbox_top','bbox_width','bbox_height'])
    df2 = pd.Series(np.full(len(np.matrix(best_results[i][0])),data1[i]))
    df1['classes'] = pd.Series(best_results[i][1])
    df1['confidences'] = pd.Series(best_results[i][2])
    df1['name_file'] = df2 
    DATA = DATA.append(df1, ignore_index = True)
  return DATA

In [0]:
df_func_annots = func_annots(txt_list, best_results_per_input)

In [0]:
df_func_annots.to_csv('/content/gdrive/My Drive/df_func_annots.csv', index = False, header=True)

In [179]:
df_func_annots

,bbox_left,bbox_top,bbox_width,bbox_height,classes,confidences,name_file
0,142.727142,141.413956,6.645656,18.443022,1,0.105840,0000249_00001_d_0000001.txt
1,253.247757,116.341927,4.338062,10.542637,1,0.109001,0000249_00001_d_0000001.txt
2,195.777939,131.615875,8.399105,16.158110,1,0.111698,0000249_00001_d_0000001.txt
3,284.914856,102.763924,4.213929,10.202283,1,0.113531,0000249_00001_d_0000001.txt
4,52.854851,73.356804,29.292780,12.229389,14,0.116739,0000249_00001_d_0000001.txt
...,...,...,...,...,...,...,...
10767,216.255920,175.699677,22.667385,24.854778,68,0.560586,0000345_07057_d_0000345.txt
10768,142.174393,217.654724,31.036476,35.812534,3,0.682732,0000345_07057_d_0000345.txt
10769,161.729187,224.124603,34.600784,40.937412,3,0.848030,0000345_07057_d_0000345.txt
10770,106.567909,207.141174,29.717857,33.303131,68,0.908687,0000345_07057_d_0000345.txt


In [0]:
from PIL import Image 
import pandas as pd
import numpy as np

In [0]:
df_size = pd.DataFrame(columns=['width','height'])
for i in range(len(image_list)):
  im = Image.open(image_list[i])
  width, height = im.size
  df_size = df_size.append({'width': width, 'height': height, 'coeff': 300/height}, ignore_index=True)

In [0]:
data1 = []
for f in txt_list:
  data1.append(f[f.rindex('/')+1:])

In [0]:
df_size['name_file'] = data1

In [0]:
df_annots_new = pd.DataFrame()

In [0]:
df_annots_new = df_annots.copy()

In [0]:
df_annots_new = df_annots_new.merge(df_size,how='left',left_on = 'name_file',right_on = 'name_file')

In [231]:
df_annots_new.head()

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff
0,879.0,483.0,10.0,31.0,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt,960.0,540.0,0.555556
1,854.0,479.0,8.0,18.0,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt,960.0,540.0,0.555556
2,620.0,445.0,63.0,35.0,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,960.0,540.0,0.555556
3,579.0,406.0,61.0,30.0,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,960.0,540.0,0.555556
4,48.0,489.0,104.0,42.0,1.0,4.0,0.0,2.0,0000249_00001_d_0000001.txt,960.0,540.0,0.555556


In [0]:
df_annots_new['bbox_left_x'] = (df_annots_new.bbox_left * df_annots_new.coeff).astype('float')
df_annots_new.bbox_top = (df_annots_new.bbox_top * df_annots_new.coeff).astype('float')
df_annots_new.bbox_width = (df_annots_new.bbox_width * df_annots_new.coeff).astype('float')
df_annots_new.bbox_height = (df_annots_new.bbox_height * df_annots_new.coeff).astype('float')
df_annots_new.width = (df_annots_new.width * df_annots_new.coeff).astype('float')
df_annots_new.height = (df_annots_new.height * df_annots_new.coeff).astype('float')

In [0]:
df_annots_new['bbox_center'] = (df_annots_new.width/2).astype('float')
df_annots_new.bbox_left = 0
df_annots_new.bbox_left = df_annots_new.bbox_center - 150
df_annots_new['bbox_right'] = df_annots_new.bbox_center + 150
df_annots_new['bbox_right_x'] = df_annots_new.bbox_left_x + df_annots_new.bbox_width

In [0]:
df_annots_new_1 = df_annots_new.copy()

In [235]:
df_annots_new_1

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff,bbox_left_x,bbox_center,bbox_right,bbox_right_x
0,116.666667,268.333333,5.555556,17.222222,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,488.333333,266.666667,416.666667,493.888889
1,116.666667,266.111111,4.444444,10.000000,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,474.444444,266.666667,416.666667,478.888889
2,116.666667,247.222222,35.000000,19.444444,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,344.444444,266.666667,416.666667,379.444444
3,116.666667,225.555556,33.888889,16.666667,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,321.666667,266.666667,416.666667,355.555556
4,116.666667,271.666667,57.777778,23.333333,1.0,4.0,0.0,2.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,26.666667,266.666667,416.666667,84.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40164,116.666667,264.166667,48.750000,21.250000,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,295.416667,266.666667,416.666667,344.166667
40165,116.666667,240.416667,42.083333,17.916667,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,435.833333,266.666667,416.666667,477.916667
40166,116.666667,241.666667,42.916667,18.750000,1.0,4.0,0.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,513.333333,266.666667,416.666667,556.250000
40167,116.666667,216.250000,22.500000,14.583333,1.0,8.0,1.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,543.750000,266.666667,416.666667,566.250000


In [0]:
df_annots_new_2 = pd.DataFrame()

In [0]:
df_annots_new_2 = df_annots_new_1.loc[(df_annots_new_1.bbox_left_x < df_annots_new_1.bbox_right) & (df_annots_new_1.bbox_right_x > df_annots_new_1.bbox_left)]

In [0]:
df_annots_new_2 = df_annots_new_1.loc[((df_annots_new_1.bbox_left_x >= df_annots_new_1.bbox_left) & (df_annots_new_1.bbox_left_x < df_annots_new_1.bbox_right) & (df_annots_new_1.bbox_right_x > df_annots_new_1.bbox_left) & (df_annots_new_1.bbox_right_x <= df_annots_new_1.bbox_right))]

In [0]:
df_annots_new_2 = pd.DataFrame()
for i in range(len(df_annots_new_1)):
  if ((df_annots_new_1.iloc[i,12]>=df_annots_new_1.iloc[i,0])&(df_annots_new_1.iloc[i,12]<df_annots_new_1.iloc[i,14])&(df_annots_new_1.iloc[i,15]>df_annots_new_1.iloc[i,0])&(df_annots_new_1.iloc[i,15]<=df_annots_new_1.iloc[i,14])):
    df_annots_new_2 = df_annots_new_2.append(df_annots_new_1.iloc[i])

In [0]:
df_annots_new_3 = df_annots_new_2.copy()

In [245]:
df_annots_new_3

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff,bbox_left_x,bbox_center,bbox_right,bbox_right_x
2,116.666667,247.222222,35.000000,19.444444,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,344.444444,266.666667,416.666667,379.444444
3,116.666667,225.555556,33.888889,16.666667,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,321.666667,266.666667,416.666667,355.555556
6,116.666667,281.111111,7.222222,11.111111,1.0,2.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,168.333333,266.666667,416.666667,175.555556
7,116.666667,282.777778,9.444444,16.666667,1.0,10.0,1.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,167.222222,266.666667,416.666667,176.666667
8,116.666667,276.666667,45.555556,23.333333,1.0,4.0,1.0,0.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,287.777778,266.666667,416.666667,333.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40160,116.666667,223.333333,5.000000,13.750000,1.0,1.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,385.000000,266.666667,416.666667,390.000000
40161,116.666667,234.583333,16.250000,10.000000,1.0,10.0,0.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,388.750000,266.666667,416.666667,405.000000
40162,116.666667,231.250000,6.250000,12.083333,1.0,2.0,0.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,394.166667,266.666667,416.666667,400.416667
40163,116.666667,245.416667,40.416667,18.750000,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,273.750000,266.666667,416.666667,314.166667


In [0]:
df_annots_new_3.bbox_right_x = df_annots_new_3.bbox_right_x - df_annots_new_3.bbox_left
df_annots_new_3.bbox_center = df_annots_new_3.bbox_center - df_annots_new_3.bbox_left
df_annots_new_3.bbox_right = df_annots_new_3.bbox_right - df_annots_new_3.bbox_left
df_annots_new_3.bbox_left_x = df_annots_new_3.bbox_left_x - df_annots_new_3.bbox_left

In [0]:
df_annots_new_3.bbox_left = df_annots_new_3.bbox_left - df_annots_new_3.bbox_left

In [248]:
df_annots_new_3

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff,bbox_left_x,bbox_center,bbox_right,bbox_right_x
2,0.0,247.222222,35.000000,19.444444,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,227.777778,150.0,300.0,262.777778
3,0.0,225.555556,33.888889,16.666667,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,205.000000,150.0,300.0,238.888889
6,0.0,281.111111,7.222222,11.111111,1.0,2.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,51.666667,150.0,300.0,58.888889
7,0.0,282.777778,9.444444,16.666667,1.0,10.0,1.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,50.555556,150.0,300.0,60.000000
8,0.0,276.666667,45.555556,23.333333,1.0,4.0,1.0,0.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,171.111111,150.0,300.0,216.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40160,0.0,223.333333,5.000000,13.750000,1.0,1.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,268.333333,150.0,300.0,273.333333
40161,0.0,234.583333,16.250000,10.000000,1.0,10.0,0.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,272.083333,150.0,300.0,288.333333
40162,0.0,231.250000,6.250000,12.083333,1.0,2.0,0.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,277.500000,150.0,300.0,283.750000
40163,0.0,245.416667,40.416667,18.750000,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,157.083333,150.0,300.0,197.500000


In [0]:
df_annots_new_4 = df_annots_new_3.copy()

In [0]:
df_annots_new_4.to_csv('/content/gdrive/My Drive/df_annots_new_4.csv', index = False, header=True)

In [0]:
df_func_annots_copy = df_func_annots.copy()

In [252]:
df_annots_new_4.head()

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff,bbox_left_x,bbox_center,bbox_right,bbox_right_x
2,0.0,247.222222,35.000000,19.444444,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,227.777778,150.0,300.0,262.777778
3,0.0,225.555556,33.888889,16.666667,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,205.000000,150.0,300.0,238.888889
6,0.0,281.111111,7.222222,11.111111,1.0,2.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,51.666667,150.0,300.0,58.888889
7,0.0,282.777778,9.444444,16.666667,1.0,10.0,1.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,50.555556,150.0,300.0,60.000000
8,0.0,276.666667,45.555556,23.333333,1.0,4.0,1.0,0.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,171.111111,150.0,300.0,216.666667


In [253]:
df_annots_new_5 = pd.DataFrame()
df_annots_new_5 = df_annots_new_4[['bbox_left_x','bbox_top','bbox_width','bbox_height','name_file','object_category']]
df_annots_new_5.head()

,bbox_left_x,bbox_top,bbox_width,bbox_height,name_file,object_category
2,227.777778,247.222222,35.000000,19.444444,0000249_00001_d_0000001.txt,4.0
3,205.000000,225.555556,33.888889,16.666667,0000249_00001_d_0000001.txt,4.0
6,51.666667,281.111111,7.222222,11.111111,0000249_00001_d_0000001.txt,2.0
7,50.555556,282.777778,9.444444,16.666667,0000249_00001_d_0000001.txt,10.0
8,171.111111,276.666667,45.555556,23.333333,0000249_00001_d_0000001.txt,4.0


In [254]:
df_func_annots_copy_1 = pd.DataFrame()
df_func_annots_copy_1 = df_func_annots_copy[['bbox_left','bbox_top','bbox_width','bbox_height','name_file','classes']]
df_func_annots_copy_1.head()

,bbox_left,bbox_top,bbox_width,bbox_height,name_file,classes
0,142.727142,141.413956,6.645656,18.443022,0000249_00001_d_0000001.txt,1
1,253.247757,116.341927,4.338062,10.542637,0000249_00001_d_0000001.txt,1
2,195.777939,131.615875,8.399105,16.158110,0000249_00001_d_0000001.txt,1
3,284.914856,102.763924,4.213929,10.202283,0000249_00001_d_0000001.txt,1
4,52.854851,73.356804,29.292780,12.229389,0000249_00001_d_0000001.txt,14


In [255]:
for i in range(len(df_annots_new_5)):
  if (df_annots_new_5.object_category.iloc[i] in np.array([5,7,8,10])):
    df_annots_new_5.object_category.iloc[i] = 'other_vehicle'
  if (df_annots_new_5.object_category.iloc[i] in np.array([1,2])):
    df_annots_new_5.object_category.iloc[i] = 'people'
  if(df_annots_new_5.object_category.iloc[i]== 3):
    df_annots_new_5.object_category.iloc[i] = 'bicycle' 
  if(df_annots_new_5.object_category.iloc[i]== 6):
    df_annots_new_5.object_category.iloc[i] = 'truck'
  if(df_annots_new_5.object_category.iloc[i]== 9):
    df_annots_new_5.object_category.iloc[i] = 'bus'
  if(df_annots_new_5.object_category.iloc[i]== 0):
    df_annots_new_5.object_category.iloc[i] = 'ignored_regions'  
  if(df_annots_new_5.object_category.iloc[i]== 4):
    df_annots_new_5.object_category.iloc[i] = 'сar'
  if(df_annots_new_5.object_category.iloc[i]== 11):
    df_annots_new_5.object_category.iloc[i] = 'other_1'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/pyt

In [256]:
a = np.array(range(4,81))
a = np.delete(a, [2,4])
a

array([ 4,  5,  7,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
       57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
       74, 75, 76, 77, 78, 79, 80])

In [257]:
for i in range(len(df_func_annots_copy_1)):
  if(df_func_annots_copy_1.classes.iloc[i] in a):
    df_func_annots_copy_1.classes.iloc[i] = 'other'
  if(df_func_annots_copy_1.classes.iloc[i]== 1):
    df_func_annots_copy_1.classes.iloc[i] = 'people'
  if(df_func_annots_copy_1.classes.iloc[i]== 2):
    df_func_annots_copy_1.classes.iloc[i] = 'bicycle'
  if(df_func_annots_copy_1.classes.iloc[i]== 8):
    df_func_annots_copy_1.classes.iloc[i] = 'truck'
  if(df_func_annots_copy_1.classes.iloc[i]== 6):
    df_func_annots_copy_1.classes.iloc[i] = 'bus'
  if(df_func_annots_copy_1.classes.iloc[i]== 3):
    df_func_annots_copy_1.classes.iloc[i] = 'сar'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoi

In [0]:
df_func_annots_copy_2 = pd.DataFrame()
df_func_annots_copy_2 = df_func_annots_copy_2.append(df_func_annots_copy_1[df_func_annots_copy_1['classes']!='other'], ignore_index=True)

In [259]:
df_func_annots_copy_2

,bbox_left,bbox_top,bbox_width,bbox_height,name_file,classes
0,142.727142,141.413956,6.645656,18.443022,0000249_00001_d_0000001.txt,people
1,253.247757,116.341927,4.338062,10.542637,0000249_00001_d_0000001.txt,people
2,195.777939,131.615875,8.399105,16.158110,0000249_00001_d_0000001.txt,people
3,284.914856,102.763924,4.213929,10.202283,0000249_00001_d_0000001.txt,people
4,103.313461,113.245995,213.426346,53.141106,0000249_00001_d_0000001.txt,people
...,...,...,...,...,...,...
8721,50.278713,122.561226,31.520309,19.881392,0000345_07057_d_0000345.txt,сar
8722,111.225395,101.722885,23.381351,18.949287,0000345_07057_d_0000345.txt,сar
8723,43.025967,221.008957,29.942232,30.303490,0000345_07057_d_0000345.txt,сar
8724,142.174393,217.654724,31.036476,35.812534,0000345_07057_d_0000345.txt,сar


In [0]:
df_annots_new_5_copy = pd.DataFrame()
df_annots_new_5_copy = df_annots_new_5_copy.append(df_annots_new_5[(df_annots_new_5['object_category']!='other_1')&(df_annots_new_5['object_category']!='other_vehicle')&(df_annots_new_5['object_category']!='ignored_regions')], ignore_index=True)

In [261]:
df_annots_new_5_copy

,bbox_left_x,bbox_top,bbox_width,bbox_height,name_file,object_category
0,227.777778,247.222222,35.000000,19.444444,0000249_00001_d_0000001.txt,сar
1,205.000000,225.555556,33.888889,16.666667,0000249_00001_d_0000001.txt,сar
2,51.666667,281.111111,7.222222,11.111111,0000249_00001_d_0000001.txt,people
3,171.111111,276.666667,45.555556,23.333333,0000249_00001_d_0000001.txt,сar
4,191.111111,259.444444,33.888889,30.000000,0000249_00001_d_0000001.txt,сar
...,...,...,...,...,...,...
22210,29.583333,235.833333,6.250000,10.833333,0000316_01201_d_0000525.txt,people
22211,268.333333,223.333333,5.000000,13.750000,0000316_01201_d_0000525.txt,people
22212,277.500000,231.250000,6.250000,12.083333,0000316_01201_d_0000525.txt,people
22213,157.083333,245.416667,40.416667,18.750000,0000316_01201_d_0000525.txt,сar


In [0]:
df_func_annots_copy_2.to_csv('/content/gdrive/My Drive/df_func_annots_copy_2.csv', index = False, header=True)

In [0]:
df_annots_new_5_copy.to_csv('/content/gdrive/My Drive/df_annots_new_5_copy.csv', index = False, header=True)

In [0]:
def max_in_list(lst):
  assert lst
  m = lst[0]
  for i in lst:
    if i > m:
      m = i
  return m


In [0]:
data1 = []
for f in txt_list:
  data1.append(f[f.rindex('/')+1:])

In [0]:
def IoU_1(out, inp):

  box_in = inp
  box_out = out
  for_return = pd.DataFrame()
  for_return_1 = []
  for_return_2 = []

  for i in range(len(box_out)):
    x_out = box_out.iloc[i,0]
    y_out = box_out.iloc[i,1]
    indx_in = []
    peresec = []
    name_out = box_out.iloc[i,4]

    for j in range(len(box_in)):
      name_in = box_in.iloc[j,4]
      if (name_out == name_in):
        width_out = box_out.iloc[i,2]
        height_out = box_out.iloc[i,3]

        width_in = box_in.iloc[j,2]
        height_in = box_in.iloc[j,3]

        x1_in = box_in.iloc[j,0]
        x2_in = box_in.iloc[j,0] + width_in
          
        y1_in = box_in.iloc[j,1]
        y2_in = box_in.iloc[j,1] - height_in
        
        x1_out = box_out.iloc[i,0]
        x2_out = box_out.iloc[i,0] + width_out
        
        y1_out = box_out.iloc[i,1]
        y2_out = box_out.iloc[i,1] - height_out
        
        xA = max(x1_out, x1_in)
        yA = max(y2_out, y2_in)
        xB = min(x2_out, x2_in)
        yB = min(y1_out, y1_in)

        w_p = xB - xA
        h_p = yB - yA

        if(w_p > 0):
          if(h_p > 0):
            interArea = w_p * h_p
          else:
            interArea = 0
        else:
          interArea = 0

        boxAArea = width_out * height_out
        boxBArea = height_in * width_in
        
        iou = interArea / float(boxAArea + boxBArea - interArea)
        indx_in.append(j)
        peresec.append(iou)

    if(peresec): 
      m_iou = max_in_list(peresec)
      for q in range(len(peresec)):
        if(peresec[q] == m_iou):
          index = indx_in[q]
          m_cl = box_in.iloc[index,5]
    else:
      peresec.append(0)
      m_iou = 0
      m_cl = 'nothing'
    
    
    for_return_1.append(m_iou)
    for_return_2.append(m_cl)

    print(i)
    print(m_iou)
    print('*****')
    
  for_return['IoU'] = pd.Series(for_return_1)
  for_return['class_in'] = pd.Series(for_return_2)
  return for_return

In [0]:
df_func_annots_copy_2_1 = df_func_annots_copy_2.tail(200)

In [0]:
df_annots_new_5_copy_1 = df_annots_new_5_copy.tail(1000)

In [0]:
import math
import matplotlib.patches as patches
import matplotlib.pyplot as plt

In [0]:
df_func_annots_copy_1.to_csv('/content/gdrive/My Drive/df_func_annots_copy_1.csv', index = False, header=True)

In [0]:
df_annots_new_5.to_csv('/content/gdrive/My Drive/df_annots_new_5.csv', index = False, header=True)

In [268]:
IOU_CL = IoU_1(df_func_annots_copy_2,df_annots_new_5_copy)

Streaming output truncated to the last 5000 lines.
*****
7060
0.0
*****
7061
0.0
*****
7062
0.0
*****
7063
0.0
*****
7064
0.0
*****
7065
0.0
*****
7066
0.0
*****
7067
0.0
*****
7068
0.0
*****
7069
0.0
*****
7070
0.0
*****
7071
0.0
*****
7072
0.0
*****
7073
0.0
*****
7074
0.0
*****
7075
0.0
*****
7076
0.0
*****
7077
0.05945039080038733
*****
7078
0.0
*****
7079
0.0
*****
7080
0.0
*****
7081
0.0
*****
7082
0.24977097064267495
*****
7083
0.0
*****
7084
0.0
*****
7085
0.0
*****
7086
0.0
*****
7087
0.0
*****
7088
0.36166525072577643
*****
7089
0.0
*****
7090
0.0
*****
7091
0.0
*****
7092
0.0
*****
7093
0.0
*****
7094
0.0
*****
7095
0.0
*****
7096
0.0
*****
7097
0.0
*****
7098
0.45653624847402274
*****
7099
0.0
*****
7100
0.4254087672016731
*****
7101
0.0
*****
7102
0.17430499783393258
*****
7103
0.0
*****
7104
0.2587898109163601
*****
7105
0.0
*****
7106
0.03913912353747378
*****
7107
0.0
*****
7108
0.0
*****
7109
0.0
*****
7110
0.0
*****
7111
0.16355101216377715
*****
7112
0.0
*****
7113
0

In [0]:
IOU_CL.to_csv('/content/gdrive/My Drive/IOU_CL.csv', index = False, header=True)

In [319]:
IOU_CL

,IoU,class_in
0,0.000000,сar
1,0.000000,сar
2,0.000000,сar
3,0.000000,сar
4,0.000000,сar
...,...,...
8721,0.111681,people
8722,0.000000,сar
8723,0.000000,сar
8724,0.000000,сar


In [320]:
df_func_annots_copy_2

,bbox_left,bbox_top,bbox_width,bbox_height,name_file,classes
0,142.727142,141.413956,6.645656,18.443022,0000249_00001_d_0000001.txt,people
1,253.247757,116.341927,4.338062,10.542637,0000249_00001_d_0000001.txt,people
2,195.777939,131.615875,8.399105,16.158110,0000249_00001_d_0000001.txt,people
3,284.914856,102.763924,4.213929,10.202283,0000249_00001_d_0000001.txt,people
4,103.313461,113.245995,213.426346,53.141106,0000249_00001_d_0000001.txt,people
...,...,...,...,...,...,...
8721,50.278713,122.561226,31.520309,19.881392,0000345_07057_d_0000345.txt,сar
8722,111.225395,101.722885,23.381351,18.949287,0000345_07057_d_0000345.txt,сar
8723,43.025967,221.008957,29.942232,30.303490,0000345_07057_d_0000345.txt,сar
8724,142.174393,217.654724,31.036476,35.812534,0000345_07057_d_0000345.txt,сar


In [0]:
Final_df = pd.DataFrame()
Final_df = df_func_annots_copy_2.copy()

In [0]:
Final_df['IoU'] = IOU_CL['IoU']

In [0]:
Final_df['class_in'] = IOU_CL['class_in']

In [0]:
Final_df.to_csv('/content/gdrive/My Drive/Final_df.csv', index = False, header=True)

In [325]:
Final_df

,bbox_left,bbox_top,bbox_width,bbox_height,name_file,classes,IoU,class_in
0,142.727142,141.413956,6.645656,18.443022,0000249_00001_d_0000001.txt,people,0.000000,сar
1,253.247757,116.341927,4.338062,10.542637,0000249_00001_d_0000001.txt,people,0.000000,сar
2,195.777939,131.615875,8.399105,16.158110,0000249_00001_d_0000001.txt,people,0.000000,сar
3,284.914856,102.763924,4.213929,10.202283,0000249_00001_d_0000001.txt,people,0.000000,сar
4,103.313461,113.245995,213.426346,53.141106,0000249_00001_d_0000001.txt,people,0.000000,сar
...,...,...,...,...,...,...,...,...
8721,50.278713,122.561226,31.520309,19.881392,0000345_07057_d_0000345.txt,сar,0.111681,people
8722,111.225395,101.722885,23.381351,18.949287,0000345_07057_d_0000345.txt,сar,0.000000,сar
8723,43.025967,221.008957,29.942232,30.303490,0000345_07057_d_0000345.txt,сar,0.000000,сar
8724,142.174393,217.654724,31.036476,35.812534,0000345_07057_d_0000345.txt,сar,0.000000,сar


In [0]:
Final_df_new = Final_df.copy()

In [327]:
Final_df_new

,bbox_left,bbox_top,bbox_width,bbox_height,name_file,classes,IoU,class_in
0,142.727142,141.413956,6.645656,18.443022,0000249_00001_d_0000001.txt,people,0.000000,сar
1,253.247757,116.341927,4.338062,10.542637,0000249_00001_d_0000001.txt,people,0.000000,сar
2,195.777939,131.615875,8.399105,16.158110,0000249_00001_d_0000001.txt,people,0.000000,сar
3,284.914856,102.763924,4.213929,10.202283,0000249_00001_d_0000001.txt,people,0.000000,сar
4,103.313461,113.245995,213.426346,53.141106,0000249_00001_d_0000001.txt,people,0.000000,сar
...,...,...,...,...,...,...,...,...
8721,50.278713,122.561226,31.520309,19.881392,0000345_07057_d_0000345.txt,сar,0.111681,people
8722,111.225395,101.722885,23.381351,18.949287,0000345_07057_d_0000345.txt,сar,0.000000,сar
8723,43.025967,221.008957,29.942232,30.303490,0000345_07057_d_0000345.txt,сar,0.000000,сar
8724,142.174393,217.654724,31.036476,35.812534,0000345_07057_d_0000345.txt,сar,0.000000,сar


In [0]:
Final_df_new['err_class'] = 0 

In [329]:
for i in range(len(Final_df_new)):
  if(Final_df_new.classes.iloc[i] != Final_df_new.class_in.iloc[i]):
    Final_df_new.err_class.iloc[i] = 0
  else:
    Final_df_new.err_class.iloc[i] = 1


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
Final_df_new_1 = Final_df_new.copy()

In [565]:
Final_df_new_1

,bbox_left,bbox_top,bbox_width,bbox_height,name_file,classes,IoU,class_in,err_class
0,142.727142,141.413956,6.645656,18.443022,0000249_00001_d_0000001.txt,people,0.000000,сar,0
1,253.247757,116.341927,4.338062,10.542637,0000249_00001_d_0000001.txt,people,0.000000,сar,0
2,195.777939,131.615875,8.399105,16.158110,0000249_00001_d_0000001.txt,people,0.000000,сar,0
3,284.914856,102.763924,4.213929,10.202283,0000249_00001_d_0000001.txt,people,0.000000,сar,0
4,103.313461,113.245995,213.426346,53.141106,0000249_00001_d_0000001.txt,people,0.000000,сar,0
...,...,...,...,...,...,...,...,...,...
8721,50.278713,122.561226,31.520309,19.881392,0000345_07057_d_0000345.txt,сar,0.111681,people,0
8722,111.225395,101.722885,23.381351,18.949287,0000345_07057_d_0000345.txt,сar,0.000000,сar,1
8723,43.025967,221.008957,29.942232,30.303490,0000345_07057_d_0000345.txt,сar,0.000000,сar,1
8724,142.174393,217.654724,31.036476,35.812534,0000345_07057_d_0000345.txt,сar,0.000000,сar,1


In [0]:
task_4_df = pd.DataFrame()
task_4_df = task_4_df.append(Final_df_new_1[Final_df_new_1['IoU']>= 0.5], ignore_index=True)

In [0]:
acc_50 = sum(task_4_df.err_class==1)/len(Final_df_new_1)

In [0]:
fail_50 = len(Final_df_new_1) - sum(task_4_df.err_class==1)

In [0]:
loss_50 = len(df_annots_new_5_copy) - sum(task_4_df.err_class==1)

In [0]:
task_4_df = pd.DataFrame()
task_4_df = task_4_df.append(Final_df_new_1[Final_df_new_1['IoU']>= 0.75], ignore_index=True)
acc_75 = sum(task_4_df.err_class==1)/len(Final_df_new_1)
fail_75 = len(Final_df_new_1) - sum(task_4_df.err_class==1)
loss_75 = len(df_annots_new_5_copy) - sum(task_4_df.err_class==1)

In [0]:
task_4_df = pd.DataFrame()
task_4_df = task_4_df.append(Final_df_new_1[Final_df_new_1['IoU']>= 0.9], ignore_index=True)
acc_90 = sum(task_4_df.err_class==1)/len(Final_df_new_1)
fail_90 = len(Final_df_new_1) - sum(task_4_df.err_class==1)
loss_90 = len(df_annots_new_5_copy) - sum(task_4_df.err_class==1)

In [0]:
TASK4 = pd.DataFrame(columns=['IoU','acc','loss','fail'])
TASK4 = TASK4.append({'IoU': 0.50, 'acc':acc_50*100, 'loss': loss_50, 'fail': fail_50}, ignore_index=True)
TASK4 = TASK4.append({'IoU': 0.75, 'acc':acc_75*100, 'loss': loss_75, 'fail': fail_75}, ignore_index=True)
TASK4 = TASK4.append({'IoU': 0.90, 'acc':acc_90*100, 'loss': loss_90, 'fail': fail_90}, ignore_index=True)

In [573]:
TASK4

,IoU,acc,loss,fail
0,0.50,0.79074,22146.0,8657.0
1,0.75,0.14898,22202.0,8713.0
2,0.90,0.01146,22214.0,8725.0


In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.5)&(Final_df_new_1['classes']=='people')], ignore_index=True)
acc_50_people = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='people'])
fail_50_people = len(Final_df_new_1[Final_df_new_1['classes']=='people']) - sum(task_5_df.err_class==1)
loss_50_people = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='people']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.75)&(Final_df_new_1['classes']=='people')], ignore_index=True)
acc_75_people = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='people'])
fail_75_people = len(Final_df_new_1[Final_df_new_1['classes']=='people']) - sum(task_5_df.err_class==1)
loss_75_people = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='people']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.90)&(Final_df_new_1['classes']=='people')], ignore_index=True)
acc_90_people = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='people'])
fail_90_people = len(Final_df_new_1[Final_df_new_1['classes']=='people']) - sum(task_5_df.err_class==1)
loss_90_people = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='people']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.5)&(Final_df_new_1['classes']=='bicycle')], ignore_index=True)
acc_50_bicycle = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='bicycle'])
fail_50_bicycle = len(Final_df_new_1[Final_df_new_1['classes']=='bicycle']) - sum(task_5_df.err_class==1)
loss_50_bicycle = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='bicycle']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.75)&(Final_df_new_1['classes']=='bicycle')], ignore_index=True)
acc_75_bicycle = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='bicycle'])
fail_75_bicycle = len(Final_df_new_1[Final_df_new_1['classes']=='bicycle']) - sum(task_5_df.err_class==1)
loss_75_bicycle = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='bicycle']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.90)&(Final_df_new_1['classes']=='bicycle')], ignore_index=True)
acc_90_bicycle = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='bicycle'])
fail_90_bicycle = len(Final_df_new_1[Final_df_new_1['classes']=='bicycle']) - sum(task_5_df.err_class==1)
loss_90_bicycle = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='bicycle']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.5)&(Final_df_new_1['classes']=='сar')], ignore_index=True)
acc_50_car = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='сar'])
fail_50_car = len(Final_df_new_1[Final_df_new_1['classes']=='сar']) - sum(task_5_df.err_class==1)
loss_50_car = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='сar']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.75)&(Final_df_new_1['classes']=='сar')], ignore_index=True)
acc_75_car = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='сar'])
fail_75_car = len(Final_df_new_1[Final_df_new_1['classes']=='сar']) - sum(task_5_df.err_class==1)
loss_75_car = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='сar']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.90)&(Final_df_new_1['classes']=='сar')], ignore_index=True)
acc_90_car = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='сar'])
fail_90_car = len(Final_df_new_1[Final_df_new_1['classes']=='сar']) - sum(task_5_df.err_class==1)
loss_90_car = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='сar']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.5)&(Final_df_new_1['classes']=='truck')], ignore_index=True)
acc_50_truck = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='truck'])
fail_50_truck = len(Final_df_new_1[Final_df_new_1['classes']=='truck']) - sum(task_5_df.err_class==1)
loss_50_truck = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='truck']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.75)&(Final_df_new_1['classes']=='truck')], ignore_index=True)
acc_75_truck = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='truck'])
fail_75_truck = len(Final_df_new_1[Final_df_new_1['classes']=='truck']) - sum(task_5_df.err_class==1)
loss_75_truck = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='truck']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.90)&(Final_df_new_1['classes']=='truck')], ignore_index=True)
acc_90_truck = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='truck'])
fail_90_truck = len(Final_df_new_1[Final_df_new_1['classes']=='truck']) - sum(task_5_df.err_class==1)
loss_90_truck = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='truck']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.5)&(Final_df_new_1['classes']=='bus')], ignore_index=True)
acc_50_bus = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='bus'])
fail_50_bus = len(Final_df_new_1[Final_df_new_1['classes']=='bus']) - sum(task_5_df.err_class==1)
loss_50_bus = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='bus']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.75)&(Final_df_new_1['classes']=='bus')], ignore_index=True)
acc_75_bus = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='bus'])
fail_75_bus = len(Final_df_new_1[Final_df_new_1['classes']=='bus']) - sum(task_5_df.err_class==1)
loss_75_bus = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='bus']) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = task_5_df.append(Final_df_new_1[(Final_df_new_1['IoU']>= 0.9)&(Final_df_new_1['classes']=='bus')], ignore_index=True)
acc_90_bus = sum(task_5_df.err_class==1)/len(Final_df_new_1[Final_df_new_1['classes']=='bus'])
fail_90_bus = len(Final_df_new_1[Final_df_new_1['classes']=='bus']) - sum(task_5_df.err_class==1)
loss_90_bus = len(df_annots_new_5_copy[df_annots_new_5_copy['object_category']=='bus']) - sum(task_5_df.err_class==1)

In [0]:
TASK5 = pd.DataFrame(columns=['class','IoU','acc','loss','fail'])
TASK5 = TASK5.append({'class':'people','IoU': 0.50, 'acc':acc_50_people*100, 'loss': loss_50_people, 'fail': fail_50_people}, ignore_index=True)
TASK5 = TASK5.append({'class':'people','IoU': 0.75, 'acc':acc_75_people*100, 'loss': loss_75_people, 'fail': fail_75_people}, ignore_index=True)
TASK5 = TASK5.append({'class':'people','IoU': 0.90, 'acc':acc_90_people*100, 'loss': loss_90_people, 'fail': fail_90_people}, ignore_index=True)
TASK5 = TASK5.append({'class':'bicycle','IoU': 0.50, 'acc':acc_50_bicycle*100, 'loss': loss_50_bicycle, 'fail': fail_50_bicycle}, ignore_index=True)
TASK5 = TASK5.append({'class':'bicycle','IoU': 0.75, 'acc':acc_75_bicycle*100, 'loss': loss_75_bicycle, 'fail': fail_75_bicycle}, ignore_index=True)
TASK5 = TASK5.append({'class':'bicycle','IoU': 0.90, 'acc':acc_90_bicycle*100, 'loss': loss_90_bicycle, 'fail': fail_90_bicycle}, ignore_index=True)
TASK5 = TASK5.append({'class':'car','IoU': 0.50, 'acc':acc_50_car*100, 'loss': loss_50_car, 'fail': fail_50_car}, ignore_index=True)
TASK5 = TASK5.append({'class':'car','IoU': 0.75, 'acc':acc_75_car*100, 'loss': loss_75_car, 'fail': fail_75_car}, ignore_index=True)
TASK5 = TASK5.append({'class':'car','IoU': 0.90, 'acc':acc_90_car*100, 'loss': loss_90_car, 'fail': fail_90_car}, ignore_index=True)
TASK5 = TASK5.append({'class':'bus','IoU': 0.50, 'acc':acc_50_bus*100, 'loss': loss_50_bus, 'fail': fail_50_bus}, ignore_index=True)
TASK5 = TASK5.append({'class':'bus','IoU': 0.75, 'acc':acc_75_bus*100, 'loss': loss_75_bus, 'fail': fail_75_bus}, ignore_index=True)
TASK5 = TASK5.append({'class':'bus','IoU': 0.90, 'acc':acc_90_bus*100, 'loss': loss_90_bus, 'fail': fail_90_bus}, ignore_index=True)
TASK5 = TASK5.append({'class':'trunc','IoU': 0.50, 'acc':acc_50_truck*100, 'loss': loss_50_truck, 'fail': fail_50_truck}, ignore_index=True)
TASK5 = TASK5.append({'class':'trunc','IoU': 0.75, 'acc':acc_75_truck*100, 'loss': loss_75_truck, 'fail': fail_75_truck}, ignore_index=True)
TASK5 = TASK5.append({'class':'trunc','IoU': 0.90, 'acc':acc_90_truck*100, 'loss': loss_90_truck, 'fail': fail_90_truck}, ignore_index=True)

In [590]:
TASK5

,class,IoU,acc,loss,fail
0,people,0.50,0.333704,9538,1792
1,people,0.75,0.111235,9542,1796
2,people,0.90,0.055617,9543,1797
3,bicycle,0.50,0.000000,843,27
4,bicycle,0.75,0.000000,843,27
5,bicycle,0.90,0.000000,843,27
6,car,0.50,0.939819,10970,6008
7,car,0.75,0.131904,11019,6057
8,car,0.90,0.000000,11027,6065
9,bus,0.50,1.063830,223,93


In [0]:
for image_idx in np.arange(130,135,1):
    fig, ax = plt.subplots(1)
    image = inputs[image_idx] / 2 + 0.5
    ax.imshow(image)
    bboxes, classes, confidences = best_results_per_input[image_idx]
    for idx in range(len(bboxes)):
        x = bboxes[idx][0]
        y = bboxes[idx][1]
        w = bboxes[idx][2]
        h = bboxes[idx][3]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    box = pd.DataFrame()
    name = data1[image_idx]
    print(name)
    box = df_annots_new_5.loc[df_annots_new_5.name_file == name]
    for idx in range(len(box)):
        x1 = box.bbox_left_x.iloc[idx]
        x2 = box.bbox_top.iloc[idx]
        x3 = box.bbox_width.iloc[idx]
        x4 = box.bbox_height.iloc[idx]
        rect1 = patches.Rectangle((x1, x2), x3, x4, linewidth=1, edgecolor='blue', facecolor='none')
        ax.add_patch(rect1)
plt.show()